In [1]:
import fdob
import fdob.processing as processing
import fdob.model as model
import exp_info as info
import benchmark

import torch
from torchvision import transforms

In [2]:
df = fdob.download_cwru("./data/cwru")

# We exclude label named 999 and 0 HP motor load condition.
df = df[(df["label"] != 999) & (df["load"] != 0)]

train_df, val_df, test_df = fdob.split_dataframe(df, 0.6, 0.2)

X_train, y_train = fdob.build_from_dataframe(train_df, 4096, 2048, False)
X_val, y_val = fdob.build_from_dataframe(val_df, 4096, 2048, False)
X_test, y_test = fdob.build_from_dataframe(test_df, 4096, 2048, False)

In [3]:
model_name = "wdcnn"

model = info.model[model_name]["model"]
sample_length = info.model[model_name]["sample_length"]
tf_data = info.model[model_name]["tf"]
tf_label = [processing.NpToTensor()]
batch_size = 32
num_workers = 1

dmodule = fdob.DatasetHandler()

dmodule.assign(
    X_train,
    y_train,
    X_val,
    y_val,
    X_test,
    y_test,
    sample_length,
    "cwru",
    transforms.Compose(tf_data),
    transforms.Compose(tf_label),
    batch_size,
    num_workers
)

dmodule.assign(
    X_train,
    y_train,
    X_val,
    y_val,
    X_test,
    y_test,
    sample_length,
    "cwru0",
    transforms.Compose([processing.AWGN(0)] + tf_data),
    transforms.Compose(tf_label),
    batch_size,
    num_workers
)

In [4]:
n_exps = 4

hparam_info = info.hparam["adam"]

hparams = fdob.log_qsample(
    hparam_info["n_params"],
    hparam_info["param_names"],
    hparam_info["lb"],
    hparam_info["ub"],
    hparam_info["reversed"],
    n_exps
)

hparams

{'lr': array([0.0001    , 0.00316228, 0.00056234, 0.01778279]),
 'beta1': array([0.999     , 0.99      , 0.9       , 0.99784557]),
 'beta2': array([0.9999    , 0.99960189, 0.99841511, 0.99369043]),
 'eps': array([1.00000000e-10, 2.68269580e-09, 7.19685673e-08, 1.93069773e-06])}

In [5]:
model_kwargs = {
    "n_classes": 10
}

opt = hparam_info["optimizer"]
opt_kwargs = {
    "lr": hparams["lr"][0],
    "betas": (hparams["beta1"][0], hparams["beta2"][0]),
    "eps": hparams["eps"][0]
}

loss = torch.nn.CrossEntropyLoss
loss_kwargs = None

seed =6464
n_gpu = 1
n_epochs = 5

result_dir = "./logs/mytest"

benchmark.train(
    dmodule.dataloaders["cwru"]["train"],
    dmodule.dataloaders["cwru"]["val"],
    model,
    model_kwargs,
    opt,
    opt_kwargs,
    loss,
    loss_kwargs,
    n_epochs,
    seed,
    n_gpu,
    result_dir
)
benchmark.test(
    dmodule.dataloaders["cwru"]["test"],
    model,
    model_kwargs,
    opt,
    opt_kwargs,
    loss,
    loss_kwargs,
    n_epochs,
    seed,
    n_gpu,
    result_dir,
    "noise-free"
)
benchmark.test(
    dmodule.dataloaders["cwru0"]["test"],
    model,
    model_kwargs,
    opt,
    opt_kwargs,
    loss,
    loss_kwargs,
    n_epochs,
    seed,
    n_gpu,
    result_dir,
    "noise"
)

/home/seongjae/anaconda3/envs/cfb/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=[1])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[1])` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/seongjae/anaconda3/envs/cfb/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /home/seongjae/Documents/FaultDiagnosisOptimizerBenchmark/logs/mytest/best_model exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type             | Params
---------------------------------------------
0 | model   | WDCNN            | 54.5 K
1 | loss_fn | Cross

Sanity Checking: 0it [00:00, ?it/s]

/home/seongjae/anaconda3/envs/cfb/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/seongjae/anaconda3/envs/cfb/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/seongjae/anaconda3/envs/cfb/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1892: PossibleUserWarning: The number of training batches (42) is smaller 

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.7432762980461121
        test_loss           1.0442636013031006
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.44254279136657715
        test_loss           1.4825063943862915
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.4825063943862915, 'test_acc': 0.44254279136657715}]